## LSTM CNN

In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd

In [3]:
import pickle

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

Embeding

In [6]:
max_features = 20000
maxlen = 20
embedding_size = 128

Convolution

In [7]:
kernel_size = 5
filters = 64
pool_size = 4

LSTM

In [8]:
lstm_output_size = 128

Training

In [9]:
batch_size = 32
epochs = 20

In [10]:
def load_ds(path):
    vecfile = open(path,'r')
    return pickle.load(vecfile)

In [11]:
ds = load_ds('./data/ann/vectors_50_50.txt')
ds.head()

,category,tweet_sec
0,1,"[268, 459, 146, 2, 470, 4493]"
1,1,"[2264, 7, 9784, 3050]"
2,0,"[80, 172, 35, 1, 14, 39, 2265, 7, 9785, 2266, ..."
3,1,"[107, 41, 21, 471, 9, 659, 5, 216, 300, 5, 7, ..."
4,1,"[562, 106, 29, 563, 29, 422]"


In [12]:
print('Pad sequences (samples x time)')
tweet_sec = sequence.pad_sequences(ds['tweet_sec'], maxlen=maxlen)
print('tweet shape:', tweet_sec.shape)

Pad sequences (samples x time)
('tweet shape:', (15306, 20))


In [13]:
def build_arch():
    print('Build model...')
    model = Sequential()
    model.add(Embedding(max_features, embedding_size, input_length=maxlen))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(lstm_output_size,
                   activation='tanh', 
                   recurrent_activation='hard_sigmoid', 
                   dropout=0.2, 
                   recurrent_dropout=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [14]:
def cross_val(tweet_sec, target, nFolds):
    kFold = StratifiedKFold(n_splits=nFolds, shuffle=True)
    scores = []
    for train, test in kFold.split(tweet_sec,target):
        model = build_arch()
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(tweet_sec[train],target[train],batch_size=batch_size,epochs=epochs,verbose=1)
        score, acc = model.evaluate(tweet_sec[test], target[test],batch_size=batch_size)
        scores.append(acc * 100)
    return scores   

In [15]:
#Training with the 100% of the data set
def train_model(tweet_sec,target):
    model = build_arch()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model.fit(tweet_sec, target,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1)
#Test ?
    
        

In [17]:
results = cross_val(tweet_sec,ds['category'],10)

Build model...
Epoch 1/20
13774/13774 [==============================] - 23s - loss: 0.3952 - acc: 0.8186    
Epoch 2/20
13774/13774 [==============================] - 21s - loss: 0.1924 - acc: 0.9302    
Epoch 3/20
13774/13774 [==============================] - 21s - loss: 0.0955 - acc: 0.9672    
Epoch 4/20
13774/13774 [==============================] - 21s - loss: 0.0482 - acc: 0.9854    
Epoch 5/20
13774/13774 [==============================] - 21s - loss: 0.0279 - acc: 0.9916    
Epoch 6/20
13774/13774 [==============================] - 21s - loss: 0.0190 - acc: 0.9937    
Epoch 7/20
13774/13774 [==============================] - 21s - loss: 0.0176 - acc: 0.9943    
Epoch 8/20
13774/13774 [==============================] - 21s - loss: 0.0111 - acc: 0.9964    
Epoch 9/20
13774/13774 [==============================] - 21s - loss: 0.0101 - acc: 0.9965    
Epoch 10/20
13774/13774 [==============================] - 21s - loss: 0.0074 - acc: 0.9976    
Epoch 11/20
13774/13774 [=========

13776/13776 [==============================] - 22s - loss: 0.4036 - acc: 0.8137    
Epoch 2/20
13776/13776 [==============================] - 21s - loss: 0.1951 - acc: 0.9278    
Epoch 3/20
13776/13776 [==============================] - 21s - loss: 0.0982 - acc: 0.9678    
Epoch 4/20
13776/13776 [==============================] - 21s - loss: 0.0490 - acc: 0.9840    
Epoch 5/20
13776/13776 [==============================] - 21s - loss: 0.0264 - acc: 0.9919    - ETA:
Epoch 6/20
13776/13776 [==============================] - 21s - loss: 0.0169 - acc: 0.9944    
Epoch 7/20
13776/13776 [==============================] - 21s - loss: 0.0109 - acc: 0.9963    
Epoch 8/20
13776/13776 [==============================] - 21s - loss: 0.0097 - acc: 0.9970    - ETA: 2s - loss: 0.0093 - acc: 
Epoch 9/20
13776/13776 [==============================] - 21s - loss: 0.0141 - acc: 0.9954    
Epoch 10/20
13776/13776 [==============================] - 21s - loss: 0.0063 - acc: 0.9975    
Epoch 11/20
13776/1377

In [18]:
print("Acc: %.2f (+/- %.2f)" %(np.mean(results), np.std(results)))

Acc: 84.71 (+/- 0.99)


In [20]:
for i in range (0,len(results)):
    print "Model %d, acc: %.2f " %(i+1,results[i])

Model 1, acc: 85.51 
Model 2, acc: 83.49 
Model 3, acc: 82.77 
Model 4, acc: 85.75 
Model 5, acc: 85.62 
Model 6, acc: 84.64 
Model 7, acc: 84.05 
Model 8, acc: 85.69 
Model 9, acc: 84.25 
Model 10, acc: 85.29 


In [23]:
#read_dictionary = np.load('./data/ann/dicFile_50_50.npy').item()
#read_dictionary

In [16]:
#Training the model
#tweet_train, tweet_test, cat_train, cat_test = train_test_split(tweet_sec, ds['category'], test_size=0.1)
#trained_model = train_model(tweet_train, cat_train, tweet_test, cat_test)
trained_model = train_model(tweet_sec,ds['category'])

Build model...
Epoch 1/20
15306/15306 [==============================] - 24s - loss: 0.3880 - acc: 0.8256    
Epoch 2/20
15306/15306 [==============================] - 24s - loss: 0.1890 - acc: 0.9304    
Epoch 3/20
15306/15306 [==============================] - 24s - loss: 0.0943 - acc: 0.9681    
Epoch 4/20
15306/15306 [==============================] - 24s - loss: 0.0494 - acc: 0.9837    
Epoch 5/20
15306/15306 [==============================] - 24s - loss: 0.0261 - acc: 0.9911    
Epoch 6/20
15306/15306 [==============================] - 24s - loss: 0.0159 - acc: 0.9944    
Epoch 7/20
15306/15306 [==============================] - 24s - loss: 0.0135 - acc: 0.9954    
Epoch 8/20
15306/15306 [==============================] - 24s - loss: 0.0123 - acc: 0.9957    
Epoch 9/20
15306/15306 [==============================] - 24s - loss: 0.0082 - acc: 0.9973    
Epoch 10/20
15306/15306 [==============================] - 24s - loss: 0.0073 - acc: 0.9971    
Epoch 11/20
15306/15306 [=========

In [17]:
model_json = trained_model.model.to_json()
with open("./data/models/lstm_cnn.json",'w') as json_file:
    json_file.write(model_json)
trained_model.model.save_weights("./data/models/lstm_cnn_weights.h5")

In [19]:
trained_model.model.save("./data/models/lstm_cnn_complete.h5")